In [1]:
!pip install langchain-community
!pip install tiktoken
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.7/408.7 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 72.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.4 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.13
    Uninstalling langchain-core-0.3.13:
      Successfully uninstalled langchain-core-0.3.13
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.4
    Uninstalling langchain-0.3.4:
      Successfully uninstalled langchain-0.3.4
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━

Dataset - https://www.kaggle.com/datasets/radek1/additional-train-data-for-llm-science-exam?select=extra_train_set.csv

In [7]:
import pandas as pd

data = pd.read_csv('/content/exam-science-llm.csv')

In [8]:
data.columns

Index(['prompt', 'C', 'E', 'D', 'B', 'A', 'answer'], dtype='object')

In [9]:
data['prompt'].value_counts()

,count
prompt,
"In relation to Eunice Fay McKenzie's career, which statement accurately reflects her most notable work?",1
"What is the main economic activity of Pierpont Township in Ashtabula County, Ohio?",1
In which family does Gudeodiscus anceyi belong?,1
"What was the recognition received by Eartha Kitt's album ""Back in Business"" at the 38th Annual Grammy Awards in 1996?",1
What happened in 1956 that resulted in the transfer of canal operation to the state-owned Suez Canal Authority?,1
...,...
What is the campus of La Jolla High School surrounded by?,1
"What is the geographical location of IFK Kalix, a Swedish football club?",1
"What is the population of Dickeyville, a village in Grant County, Wisconsin?",1


Vector Store - These are like special filing cabinets for the number patterns (vectors) created by embedding models. When you want to find similar text later, the vector store helps you quickly find vectors that are close to each other. It's like organizing books by topic, but with numbers instead.

In [15]:
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from dotenv import load_dotenv
import os

load_dotenv()

os.environ["OPENAI_API_KEY"] = 'Insert Your API KEY'

# Load the document, split it into chunks, embed each chunk and load it into the vector store.
# Insert question & answer csv file
raw_documents = TextLoader('/content/exam-science-llm.csv').load()
text_splitter = CharacterTextSplitter(chunk_size=200, chunk_overlap=0, separator = "\n",)
documents = text_splitter.split_documents(raw_documents)
db = FAISS.from_documents(documents, OpenAIEmbeddings())

In [16]:
query = "In relation to Eunice Fay McKenzie's career, which statement accurately reflects her most notable work?"
docs = db.similarity_search(query)
print(docs[0].page_content)

"In relation to Eunice Fay McKenzie's career, which statement accurately reflects her most notable work?",McKenzie gained recognition for her role as a child actress in a series of iconic silent films.,"McKenzie's successful career in sound films continued into adulthood, becoming known for her versatile acting abilities.",McKenzie's collaborations with director Blake Edwards were instrumental in her rise to fame.,McKenzie is primarily remembered for her starring roles opposite Gene Autry in popular Western films of the 1940s.,"McKenzie showcased her singing talents in numerous musical productions, garnering critical acclaim.",B


In [18]:
print(documents[1])

page_content='"In relation to Eunice Fay McKenzie's career, which statement accurately reflects her most notable work?",McKenzie gained recognition for her role as a child actress in a series of iconic silent films.,"McKenzie's successful career in sound films continued into adulthood, becoming known for her versatile acting abilities.",McKenzie's collaborations with director Blake Edwards were instrumental in her rise to fame.,McKenzie is primarily remembered for her starring roles opposite Gene Autry in popular Western films of the 1940s.,"McKenzie showcased her singing talents in numerous musical productions, garnering critical acclaim.",B' metadata={'source': '/content/exam-science-llm.csv'}


Retrievers - These are like smart librarians. When you ask a question, they go to the vector store, find the most relevant information, and bring it back. They help fetch the most useful pieces of information based on what you're looking for.

In [19]:
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI

retriever = db.as_retriever()

In [20]:
qa = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=retriever)

query = "What big impact in Eunice Fay McKenzie's career?"
qa.run(query)

<ipython-input-20-c2f5dd76ef00>:1: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  qa = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=retriever)
<ipython-input-20-c2f5dd76ef00>:4: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  qa.run(query)


" McKenzie's collaborations with director Blake Edwards were instrumental in her rise to fame."

In [21]:
query = "In which family does Gudeodiscus anceyi belong?"
qa.run(query)

' Pupillidae'

In [22]:
query = "What is the geographical location of IFK Kalix, a Swedish football club? What's the answer?"
qa.run(query)

' IFK Kalix is located in Norrbotten County, in the region of Norrland, Sweden.'

In [24]:
query = "Which of the following statements accurately describes Ray Montgomerie's football career?"
qa.run(query)

' Ray Montgomerie is a former footballer who played as a defender for Dumbarton, Kilmarnock, and Partick Thistle.'

In [26]:
query = "What is the genre of the 2021 film Dreamcatcher directed by Jacob Johnson?"
qa.run(query)

" I don't know."